In [1]:
from xlrd import open_workbook

import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import time

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [4]:
cal_excel = open_workbook("/Users/roberto/Documents/Telecomm/2020/Calendarios/Calendario_Nave_Febrero 2020.xlsm")
hoja_cal = cal_excel.sheet_by_name("Febrero")

In [5]:
filas_fecha_dia = [5 + i*5 for i in range(5)]
filas_act_mx3   = [7 + i*5 for i in range(5)]
filas_act_mx2   = [9 + i*5 for i in range(5)]

cols = [2 + i for i in range(7)]

lista_act_mx2 = []
for i in filas_act_mx2:
    for j in cols:
        lista_act_mx2.append(hoja_cal.cell_value(i,j))
        
lista_act_mx3 = []
for i in filas_act_mx3:
    for j in cols:
        lista_act_mx3.append(hoja_cal.cell_value(i,j))
        
lista_dias = []
for i in filas_fecha_dia:
    for j in cols:
        lista_dias.append(hoja_cal.cell_value(i,j))
        
dict_act = {int(dia):{"mx2":mx2.split("\n"), "mx3":mx3.split("\n")}
            for dia, mx2, mx3 in zip(*[lista_dias, lista_act_mx2, lista_act_mx3]) if isinstance(dia, float)}

notas_por_dia = []
for k in dict_act:
    ev2 = '\n'.join(dict_act[k]['mx2'])
    ev3 = '\n'.join(dict_act[k]['mx3'])
    notas_por_dia.append(f"Events mx2:\n\n{ev2}\n\nEvents mx3:\n\n{ev3}")

mes_actual = datetime.datetime.now().month
año_actual = datetime.datetime.now().year
fechas = [datetime.datetime(año_actual, mes_actual, dia) for dia in range(1, len(notas_por_dia)+1)]

ValueError: day is out of range for month

In [8]:
len(notas_por_dia)

35

In [157]:
def envia_evento_google_cal(nota, fecha_inicio, fecha_final, laboral=False):
    
    autor = {"self":True, "displayName":"Roberto Cadena Vega", "email":"r.cadena.vega@gmail.com"}
    inicio = {"dateTime":fecha_inicio.isoformat(), "timeZone":"America/Mexico_City"}
    final  = {"dateTime":fecha_final.isoformat(), "timeZone":"America/Mexico_City"}
    ubicacion = "SCT MEXSAT, Telecomunicaciones 21, Área Federal Complejo de Telefonía, Iztapalapa, CDMX, Mexico"
    notificacion = {"overrides":[{"minutes":3*60, "method":"popup"}, {"minutes":15, "method":"popup"}],
                    "useDefault":False}
    
    if laboral:
        cuerpo = {"start":inicio, "end":final, "location":ubicacion,
                  "summary":"Trabajo", "description":nota, "creator":autor, "reminders":notificacion}
    else:
        cuerpo = {"start":inicio, "end":final, "location":ubicacion,
                  "summary":"Dia libre", "description":nota, "creator":autor}
    
    # If modifying these scopes, delete the file token.pickle.
    SCOPES = ['https://www.googleapis.com/auth/calendar']
    
    path_creds = '/Users/roberto/.google_cal/credentials.json'
    path_token = '/Users/roberto/.google_cal/token.pickle'
    
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists(path_token):
        with open(path_token, 'rb') as token:
            creds = pickle.load(token)
            
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(path_creds, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open(path_token, 'wb') as token:
            pickle.dump(creds, token)

    service = build('calendar', 'v3', credentials=creds)

    evento_creado = service.events().insert(calendarId="gokp61pd7nfarmlhv6el6eq2jo@group.calendar.google.com",
                                            body=cuerpo).execute()
    #service.events().insert(calendarId="gokp61pd7nfarmlhv6el6eq2jo@group.calendar.google.com", body=)
    return evento_creado

In [160]:
respuestas = []

for nota, fecha in zip(*[notas_por_dia[24:], fechas[24:]]):
    if fecha.isoweekday() > 5:
        respuestas.append(envia_evento_google_cal(nota=nota, 
                                    fecha_inicio=fecha+datetime.timedelta(hours=8),
                                    fecha_final=fecha+datetime.timedelta(hours=8.5)))
    else:
        respuestas.append(envia_evento_google_cal(nota=nota, 
                                    fecha_inicio=fecha+datetime.timedelta(hours=8),
                                    fecha_final=fecha+datetime.timedelta(hours=17.5), laboral=True))
    time.sleep(60)

In [161]:
respuestas

[{'kind': 'calendar#event',
  'etag': '"3156398177954000"',
  'id': 'q9mnaeusa8l11de8s4lrvubffc',
  'status': 'confirmed',
  'htmlLink': 'https://www.google.com/calendar/event?eid=cTltbmFldXNhOGwxMWRlOHM0bHJ2dWJmZmMgZ29rcDYxcGQ3bmZhcm1saHY2ZWw2ZXEyam9AZw',
  'created': '2020-01-05T04:38:08.000Z',
  'updated': '2020-01-05T04:38:08.977Z',
  'summary': 'Dia libre',
  'description': 'Events mx2:\n\nRanging Both Sides\n\nEvents mx3:\n\nLunar Eclipse\nRanging Both Sides',
  'location': 'SCT MEXSAT, Telecomunicaciones 21, Área Federal Complejo de Telefonía, Iztapalapa, CDMX, Mexico',
  'creator': {'email': 'r.cadena.vega@gmail.com'},
  'organizer': {'email': 'gokp61pd7nfarmlhv6el6eq2jo@group.calendar.google.com',
   'displayName': 'Telecomm',
   'self': True},
  'start': {'dateTime': '2020-01-25T08:00:00-06:00',
   'timeZone': 'America/Mexico_City'},
  'end': {'dateTime': '2020-01-25T08:30:00-06:00',
   'timeZone': 'America/Mexico_City'},
  'iCalUID': 'q9mnaeusa8l11de8s4lrvubffc@google.com',
